In [2]:
from LowPrecisionApproxGP.model.inducing_point_kernel import VarPrecisionInducingPointKernel
import gpytorch
# from gpytorch.kernels.inducing_point_kernel import InducingPointKernel
import torch
from LowPrecisionApproxGP.util.GreedyTrain import greedy_train

In [2]:
# Define Model
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = VarPrecisionInducingPointKernel(
            gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel()), inducing_points=torch.empty(1), likelihood=likelihood, dtype=torch.float16)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)



In [3]:
import math
# Create / Reshape Data
train_x = torch.linspace(0, 1, 100).reshape(-1,1)
train_y = torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * math.sqrt(0.04)
train_y = train_y.reshape(-1,1)

# Set up inducing point variables
inducing_point_candidates = train_x.detach().clone()
training_iter = 1000
max_inducing_points = 1000

In [1]:
# Create Likelihood / Model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood,model)

# Set to training mode
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters
# Train Model
model = greedy_train((train_x,train_y), model, mll,max_inducing_points,training_iter,dtype=torch.float16,Use_Max=True,J=20,max_Js=10)

NameError: name 'gpytorch' is not defined

In [5]:
model.covar_module.inducing_points

Parameter containing:
tensor([[0.4848],
        [0.5253],
        [0.4949],
        [0.5455]])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_x.numpy(),train_y.numpy())